# Assignment 3 on Natural Language Processing

## Date : 30th Sept, 2020

### Instructor : Prof. Sudeshna Sarkar

### Teaching Assistants : Alapan Kuila, Aniruddha Roy, Anusha Potnuru, Uppada Vishnu

### Name : Kanishk Singh

### Roll : 17CS30018

The central idea of this assignment is to use Naive Bayes classifier and LSTM based classifier and compare the models by accuracy on IMDB dataset.



Please submit with outputs. 

In [1]:
import re
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import keras
from sklearn.metrics import classification_report

Using TensorFlow backend.


In [2]:
#Load the IMDB dataset. You can load it using pandas as dataframe
import os
path = os.path.abspath('IMDB-Dataset.csv')  #Make sure you have uploaded the datafile.
df = pd.DataFrame()
df = pd.read_csv(path)
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


# Preprocessing
PrePrecessing that needs to be done on lower cased corpus

1. Remove html tags
2. Remove URLS
3. Remove non alphanumeric character
4. Remove Stopwords
5. Perform stemming and lemmatization

You can use regex from re. 

In [3]:
import nltk

nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
stopword = set(stopwords.words('english'))

from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()


from nltk.stem import PorterStemmer
ps = PorterStemmer()

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/kanishk/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/kanishk/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
def remove_html_tags(text):
    TAG_RE = re.compile(r'<[^>]+>')
    return TAG_RE.sub('', text)

def remove_URLS(text):
    text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    return text

def remove_stopwords(text):
    word_tokens = word_tokenize(text)
    filtered_sentence = [w for w in word_tokens if not w in stopword]
    sent = " ".join(filtered_sentence)
    return sent

def remove_alphanumeric(text):
    text = re.sub(r'[^a-zA-Z]', " ", text)
    return text

def lemmatize_and_stemm(text):
    word_tokens = word_tokenize(text)
    sent = [lemmatizer.lemmatize(w,"v") for w in word_tokens]
    result = " ".join(sent)
    return result

def remove_punct(data=None):
    punctuations = '''!()-[]{};:'"\,”<>.“/?@#$%^&*_~'''
    for x in data.lower(): 
        if x in punctuations: 
            data = data.replace(x, "")
    return data

def text_preprocess(text):
    text = remove_html_tags(text)
    text = remove_URLS(text)
    text = remove_punct(text)
    text = remove_stopwords(text)
    text = remove_alphanumeric(text)
    text = lemmatize_and_stemm(text)
    return text
    

In [5]:
for index, row in df.iterrows():
    df.loc[index] = [text_preprocess(row[0]),row[1]]
df.head()

,review,sentiment
0,One reviewers mention watch Oz episode youll h...,positive
1,A wonderful little production The film techniq...,positive
2,I think wonderful way spend time hot summer we...,positive
3,Basically theres family little boy Jake think ...,negative
4,Petter Matteis Love Time Money visually stun f...,positive


In [6]:
# Print Statistics of Data like avg length of sentence , proposition of data w.r.t class labels
sent_count = 0
word_count = 0
pos_class = 0
neg_class = 0
for index, row in df.iterrows():
    word_tokens = word_tokenize(row[0])
    word_count  = word_count + len(word_tokens)
    sent_count += 1
    if row[1]=='positive':
        pos_class += 1
    else:
        neg_class += 1

print("Average length of sentence = " ,word_count/sent_count," words.")
print("Total number of examples with positive sentiment = ",pos_class," and with negative sentiment = ",neg_class)

Average length of sentence =  128.5395  words.
Total number of examples with positive sentiment =  25000  and with negative sentiment =  25000


# Naive Bayes classifier

In [7]:
# get reviews column from df
reviews = df["review"].tolist()

# get labels column from df
labels = df["sentiment"].tolist()

In [10]:
# Use label encoder to encode labels. Convert to 0/1
encoder = LabelEncoder()
encoded_labels = encoder.fit_transform(labels)

# print(enc.classes_)

In [11]:
train_sentences, test_sentences, train_labels, test_labels = train_test_split(reviews, encoded_labels, test_size=0.2,stratify=encoded_labels)
# Split the data into train and test (80% - 20%). 
# Use stratify in train_test_split so that both train and test have similar ratio of positive and negative samples.

# train_sentences, test_sentences, train_labels, test_labels

Here there are two approaches possible for building vocabulary for the naive Bayes.
1. Take the whole data (train + test) to build the vocab. In this way while testing there is no word which will be out of vocabulary.
2. Take the train data to build vocab. In this case, some words from the test set may not be in vocab and hence one needs to perform smoothing so that one the probability term is not zero.
 
You are supposed to go by the 2nd approach.
 
Also building vocab by taking all words in the train set is memory intensive, hence you are required to build vocab by choosing the top 2000 - 3000 frequent words in the training corpus.

> $ P(x_i | w_j) = \frac{ N_{x_i,w_j}\, +\, \alpha }{ N_{w_j}\, +\, \alpha*d} $


$N_{x_i,w_j}$ : Number of times feature $x_i$ appears in samples of class $w_j$

$N_{w_j}$ : Total count of features in class $w_j$

$\alpha$ : Parameter for additive smoothing. Here consider $\alpha$ = 1

$d$ : Dimentionality of the feature vector  $x = [x_1,x_2,...,x_d]$. In our case its the vocab size.






In [12]:
from sklearn.feature_extraction.text import CountVectorizer
# Use Count vectorizer to get frequency of the words
'''
max_features parameter : If not None, build a vocabulary that only consider the top max_features ordered by term frequency across the corpus.
vec = CountVectorizer(max_features = 3000)
X = vec.fit_transform(Sentence_list)
'''

train_pos = []
train_neg = []

for i in range(len(train_sentences)):
    if train_labels[i]==1:
        train_pos.append(train_sentences[i])
    else:
        train_neg.append(train_sentences[i])

##VOCABULARY FOR POSITIVE CLASS
cv_pos = CountVectorizer(max_features = 3000)   
cv_pos_fit=cv_pos.fit_transform(train_pos)    
word_list_pos = cv_pos.get_feature_names();    
count_list_pos = cv_pos_fit.toarray().sum(axis=0)   
pos_dict = dict(zip(word_list_pos,count_list_pos))

##VOCABULARY FOR NEGATIVE CLASS
cv_neg = CountVectorizer(max_features = 3000)   
cv_neg_fit=cv_neg.fit_transform(train_neg)    
word_list_neg = cv_neg.get_feature_names();    
count_list_neg = cv_neg_fit.toarray().sum(axis=0)   
neg_dict = dict(zip(word_list_neg,count_list_neg))

In [13]:
pos_dict_total = count_list_pos.sum(axis = 0)
neg_dict_total = count_list_neg.sum(axis = 0)
print(pos_dict_total," ",neg_dict_total)

1972639   1988468


In [16]:
# Use laplace smoothing for words in test set not present in vocab of train set
def prob(word,class_dict,total):
    num = 0
    if word in class_dict.keys():
        num = class_dict[word] + 1
    else:
        num = 1
    dem = total + len(class_dict)
    return num,dem

In [17]:
# Build the model. Don't use the model from sklearn
import math
def predict(sent):
    sent = word_tokenize(sent)
    pos_num = 0
    pos_dem = 0
    neg_num = 0
    neg_dem = 0
    for word in sent:
        num,dem  = prob(word,pos_dict,pos_dict_total)
        pos_num += math.log(num)
        pos_dem += math.log(dem)
        
        num,dem  = prob(word,neg_dict,neg_dict_total)
        neg_num += math.log(num)
        neg_dem += math.log(dem)
    
    if pos_num-pos_dem>neg_num-neg_dem:
        return(1)
    else:
        return(0)
        


In [18]:
# Test the model on test set and report Accuracy
def accuracy(test_sentences,test_labels):
    count = 0
    for index in range(len(test_sentences)):
        sent = test_sentences[index]
        trgt = test_labels[index]
        pred = predict(sent)
        if pred==trgt:
            count += 1
    acc = count/len(test_labels)
    acc *= 100
    return count,len(test_labels),acc

correct,total, acc = accuracy(test_sentences,test_labels)
print("Accuracy for naive bayes = ",acc, " classifiying (",correct,"/",total,")examples.")


Accuracy for naive bayes =  81.41000000000001  classifiying ( 8141 / 10000 )examples.


# *LSTM* based Classifier

Use the above train and test splits.

In [19]:
# Hyperparameters of the model
vocab_size = 3000# choose based on statistics
oov_tok = '<OOK>'
embedding_dim = 100
max_length = 130# choose based on statistics, for example 150 to 200
padding_type='post'
trunc_type='post'

In [20]:
#train_sentences, test_sentences, train_labels, test_labels = train_test_split(reviews, encoded_labels, test_size=0.2,stratify=encoded_labels)

# tokenize sentences
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index

# convert train dataset to sequence and pad sequences
train_sequences = tokenizer.texts_to_sequences(train_sentences)
train_padded = pad_sequences(train_sequences, padding='post', maxlen=max_length)

# convert Test dataset to sequence and pad sequences
test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_sequences, padding='post', maxlen=max_length)

In [21]:
# model initialization
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),
    keras.layers.Dense(24, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

# compile model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# model summary
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 130, 100)          300000    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128)               84480     
_________________________________________________________________
dense_1 (Dense)              (None, 24)                3096      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 25        
Total params: 387,601
Trainable params: 387,601
Non-trainable params: 0
_________________________________________________________________


In [22]:
num_epochs = 5
history = model.fit(train_padded, train_labels, 
                    epochs=num_epochs, verbose=1, 
                    validation_split=0.1)

/home/kanishk/miniconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 36000 samples, validate on 4000 samples
Epoch 1/5
36000/36000 [==============================] - 118s 3ms/step - loss: 0.4004 - accuracy: 0.8211 - val_loss: 0.3022 - val_accuracy: 0.8758
Epoch 2/5
36000/36000 [==============================] - 113s 3ms/step - loss: 0.2952 - accuracy: 0.8785 - val_loss: 0.3023 - val_accuracy: 0.8730
Epoch 3/5
36000/36000 [==============================] - 113s 3ms/step - loss: 0.2579 - accuracy: 0.8958 - val_loss: 0.3216 - val_accuracy: 0.8660
Epoch 4/5
36000/36000 [==============================] - 113s 3ms/step - loss: 0.2207 - accuracy: 0.9135 - val_loss: 0.3595 - val_accuracy: 0.8665
Epoch 5/5
36000/36000 [==============================] - 113s 3ms/step - loss: 0.1897 - accuracy: 0.9263 - val_loss: 0.3511 - val_accuracy: 0.8625


In [23]:
# Calculate accuracy on Test data
'''
prediction = model.predict(test_padded)

'''

# Get probabilities
prediction = model.predict(test_padded)

# Get labels based on probability 1 if p>= 0.5 else 0
test_preds = [p>=0.5 for p in prediction]

# Accuracy : one can use classification_report from sklearn
target_names = ['negative', 'positive']
print(classification_report(test_labels, test_preds, target_names=target_names))

              precision    recall  f1-score   support

    negative       0.86      0.86      0.86      5000
    positive       0.86      0.86      0.86      5000

    accuracy                           0.86     10000
   macro avg       0.86      0.86      0.86     10000
weighted avg       0.86      0.86      0.86     10000



## Get predictions for random examples

In [24]:
# reviews on which we need to predict
sentence = ["The movie was very touching and heart whelming", 
            "I have never seen a terrible movie like this", 
            "the movie plot is terrible but it had good acting"]

# convert to a sequence
sequences = tokenizer.texts_to_sequences(sentence)

# pad the sequence
padded = pad_sequences(sequences, padding='post', maxlen=max_length) 

# Get probabilities
print(model.predict(padded))
preds = [p>=0.5 for p in model.predict(padded)]

# Get labels based on probability 1 if p>= 0.5 else 0
for i in range(len(preds)):
    if preds[i]==True:
        print("The sentiment predicted is positive for the review = "+sentence[i])
    else:
        print("The sentiment predicted is negative for the review = "+sentence[i])



[[0.7647391 ]
 [0.10442615]
 [0.05545776]]
The sentiment predicted is positive for the review = The movie was very touching and heart whelming
The sentiment predicted is negative for the review = I have never seen a terrible movie like this
The sentiment predicted is negative for the review = the movie plot is terrible but it had good acting
